In [ ]:


%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from datetime import timedelta
import parcels as pc


from parcels.fieldset import *  # noqa
from parcels.particle import *  # noqa
from parcels.particleset import *  # noqa
from parcels.field import *  # noqa
from parcels.kernel import *  # noqa
import parcels.rng as ParcelsRandom  # noqa
from parcels.particlefile import *  # noqa
from parcels.kernels import *  # noqa
from parcels.scripts import *  # noqa
from parcels.gridset import *  # noqa
from parcels.grid import *  # noqa
from parcels.tools import *  # noqa

In [ ]:
#Deffining the File
filenames = {'U' :"jacob.nc",
            'V' : "jacob.nc"}

#Defining the Variables 
variables = {'U': 'u',
             'V': 'v'}

dimensions = {'time':'time',
              'lat': 'y',
              'lon': 'x'}

#Importing the data 
fieldset = FieldSet.from_netcdf("jacob.nc", variables, dimensions, mesh='flat')


In [ ]:
#defining particle set locations (repeated launch every day)

psetAPP = ParticleSet.from_line(fieldset=fieldset, size=4, pclass=JITParticle, start=(-2.1e6,0.6e6),finish=(-2e6,-0.2e6), repeatdt=timedelta(days=1))
psetAmund1 = ParticleSet.from_line(fieldset=fieldset, size=3, pclass=JITParticle, start=(-1.7e6,-0.4e6),finish=(-1.7e6,-1.1e6), repeatdt=timedelta(days=1))
psetAmund2 = ParticleSet.from_line(fieldset=fieldset, size=5, pclass=JITParticle, start=(-1.5,-1.2e6),finish=(-0.8e6,-1.3e6), repeatdt=timedelta(days=1))
psetRoss = ParticleSet.from_line(fieldset=fieldset, size=5, pclass=JITParticle, start=(-0.1,-1.3e6),finish=(0.2e6,-1.3e6), repeatdt=timedelta(days=1))
psetAAP = ParticleSet.from_line(fieldset=fieldset, size=1, pclass=JITParticle, start=(1.2e6,-2.2e6),finish=(1.2e6,-2.21e6), repeatdt=timedelta(days=1))
psetWed = ParticleSet.from_line(fieldset=fieldset, size=7, pclass=JITParticle, start=(-1.2,1e6),finish=(0.9e6,1e6), repeatdt=timedelta(days=1))
psetFim = ParticleSet.from_line(fieldset=fieldset, size=3, pclass=JITParticle, start=(-0.1e6,2.15e6),finish=(0.5e6,2.14e6), repeatdt=timedelta(days=1))
psetWilk = ParticleSet.from_line(fieldset=fieldset, size=2, pclass=JITParticle, start=(1.9e6,-2e6),finish=(2e6,-1.9e6), repeatdt=timedelta(days=1))
psetEast = ParticleSet.from_line(fieldset=fieldset, size=3, pclass=JITParticle, start=(2.5e6,0),finish=(2.5e6,-1e6), repeatdt=timedelta(days=1))


In [ ]:
#creating lists so can execute on all data sets 
psetlist = [  psetAmund2, psetRoss, psetAAP, psetWed, psetFim, psetWilk, psetEast]
outputlist = [ 'Amundson2.nc','Ross.nc', 'AAPeninsula.nc','Weddell.nc','Fimbul.nc','Wilkes.nc','East.nc']

In [ ]:
#execution of code

for i in range(len(psetlist)):
    pset = psetlist[i]
    output = pset.ParticleFile(name= outputlist[i], convert_at_end=False, outputdt=timedelta(hours= 24))#the output file will be updated every "24hrs" at the same time as the code is running 
    
    pset.execute(AdvectionRK4,#Using a fourth order Runge Kutter method
             runtime=timedelta(days=2191),#run for the full length of the data set 
             dt=timedelta(hours=1),#it will timestep every hour
              moviedt=timedelta(days=1),    #outputs as an animatable format (this results in an output graph every day as the code is running), Even when removed code takes around 8hrs          
             output_file=output,
             #recovery #have added a catch to delete particles when out of bounds but onedrive glitched and I'm left with an older version of my file 
             )